# Sudoku Project
by Finn Potason (s2.wang@vu.nl)

This is a quick tutorial for MSc Logic students and those who struggle to kick off the project.

First of all you need the Pycosat solver [1].
In your terminal [2], type: pip install pycosat

[1] See more details at: https://pypi.python.org/pypi/pycosat
[2] Press ctrl + alt + t.

Next we test if the solver is working: 

In [1]:
import pycosat
# cnf = [[1, 5,4], [-1,5,3,4], [-3, -4]]
# pycosat.solve(cnf)

Next, you can print all the possible assignments to this CNF. 

In [2]:
# print ('There are in total ', len(list(pycosat.itersolve(cnf))), ' possible assignments:')
# for sol in pycosat.itersolve(cnf):
#     print(sol)

## Encode Sudoku
Take a random sudoku and think what the propositional variables are.
PS: You will need the library numpy. If you don't have it, install it (and reboot if needed).

In [3]:
import numpy as np
# s_test = np.array([[8,0,6,5,0,0,0,0,0],
#                     [0,0,4,0,0,0,0,0,8],
#                     [0,0,0,0,0,0,6,0,0],
#                     [0,0,0,0,0,0,0,0,0],
#                     [3,7,0,4,5,0,0,0,0],
#                     [5,0,1,0,9,8,0,0,7],
#                     [0,0,0,0,0,7,0,2,0],
#                     [2,5,7,1,6,0,0,0,9],
#                     [0,8,0,0,3,0,0,4,0]])
# print(s_test)

In [4]:
# names = np.zeros([9,9,9], dtype = np.int)

# index = 1
# for i in range(9):
#     for j in range(9):
#         print('introduce a 9 propositional variables to each cell:')
#         for k in range(9):
#             names[i][j][k] = i * 81 + j * 9 + k +1
#             print('if the cell at row: ', i+1, ' column: ', j+1, 'is ', k+1, 'then it is named ', names[i][j][k])

In [5]:
# print ('When the cell at row 3 column 4 is 6')
# print('the variable' , names[2][3][5], ' is true')
# print('please be careful about the indexing!!!')

In [6]:
# encode_cnf = []
# print('start populate this cnf:', encode_cnf)

Next, we define two functions to obtain the function exactly one. 

In [7]:
def encode_at_most_one(names):
    encode = []
    for i in range(len(names)):
        for j in range(i+1, len(names)):
            arr = [-1*names[i], -1*names[j]]
            encode.insert(0, arr)
    return encode
      
# test_names = [1,2,3]
# enc = encode_at_most_one(test_names)
# print('at most one is encoded as: ',enc)
# encode_cnf.extend(enc)

def encode_at_least_one(names):
    return [names]
# enc = encode_at_least_one(test_names)
# print('at least one is encoded as:', enc)
# encode_cnf.extend(enc)
# print('put them together, you get: ', encode_cnf)
# print ('as you can see, there is exactly one true:')
# pycosat.solve(encode_cnf)
# TODO: introduce a function called encode_exactly_one(names) 
#       and update the code above

## 1) For each cell, exactly one variable is true.

In [8]:
def get_cell_names(n):
    names = np.zeros([n,n,n], dtype = np.int)

    index = 1
    for i in range(n):
        for j in range(n):
            for k in range(n):
                names[i][j][k] = i * n**2 + j * n + k +1
    return names

In [9]:
# n = 4
# names = get_cell_names(n)
# print(names)

In [10]:
def encode_exactly_one(names):
    enc = encode_at_most_one(names)
    enc.extend(encode_at_least_one(names))
    return enc

def encode_exactly_ones(names):
    n=len(names)
    possible_enc = []
    for i in range(n):
        for j in range(n):
            same_values = []
            for k in range(n):
                same_values.append(int(names[i][j][k]))
            possible_enc.extend(encode_exactly_one(same_values))
    return possible_enc
    

## 2) 

In [11]:
def encode_row(names):
    n=len(names)
    possible_enc = []
    for i in range(n):
        for k in range(n):
            same_values = []
            for j in range(n):
                same_values.append(int(names[i][j][k]))
            possible_enc.extend(encode_exactly_one(same_values))
    return possible_enc

## 3) For each column, for each number, exactly one variable is true.

In [12]:
def encode_column(names):
    n=len(names)
    possible_enc = []
    for k in range(n):
        for j in range(n):
            same_values = []
            for i in range(n):
                same_values.append(int(names[i][j][k]))
            possible_enc.extend(encode_exactly_one(same_values))
    return possible_enc

## 4) For each block, for each number, exactly one variable is true.

In [13]:
import math

def encode_block(names):
    n=len(names)
    block_n = int(math.sqrt(n))

    possible_enc = []
    for i in range(block_n):
        for k in range(n):

            for nj in range(block_n):
                same_values = []
                for j in range(block_n):

                    for ni in range(block_n):
                        same_values.append(int(names[i*block_n+ni][j+block_n*nj][k]))

                possible_enc.extend(encode_exactly_one(same_values))
    return possible_enc

## 5) Now put them together and see if your solver returns an assignment

In [14]:
import itertools
def remove_duplicates(cnf):
    cnf.sort()
    return list(rule for rule,_ in itertools.groupby(cnf))

In [15]:
def encode_sudoku(names, c=[]):
    c.extend(encode_exactly_ones(names))
    c.extend(encode_row(names))
    c.extend(encode_column(names))
    c.extend(encode_block(names))
    
    print('Amount of rules', len(c))
    c = remove_duplicates(c)
    print('Amount of unique rules',len(c))

    return c

## 6) What does this assignment tell you? Can you decode it and print the solution (mind the indices)?

In [16]:
def name_to_index(name):
    name-=1
    
    n=len(names)

    index_i = name // n**2
    name -= (index_i*n**2)
    index_j = name // n
    name -= (index_j*n)
    index_k = name
    
    return index_i, index_j, index_k

In [17]:
def decode_sudoku(solution):
    if solution=='UNSAT':
        raise NotImplementedError("UNSAT is not yet implemented")
        
    # sqrt with 3
    n = int(round(len(solution)**(1./3.)))

    sud = np.zeros([n,n], dtype = np.int)

    for sol in solution:
        if sol>0:
            i,j,k = name_to_index(sol)
            sud[i][j]=k+1
    
    return sud

In [18]:
# n=4
# names = get_cell_names(n)

# input_enc=[[1],[6],[11]] # 1 at first position and 2 at second
# cnf = encode_sudoku(names, input_enc)

# # solution = pycosat.solve(cnf)
# # print(decode_sudoku(solution))


# solutions = list(pycosat.itersolve(cnf))

# print ('There are in total ', len(solutions), ' possible assignments:')
# for sol in solutions[:3]:
#     print(decode_sudoku(sol))


In [19]:
from sudoku_generator import generate_puzzle

Runtime is 0.8725311756134033 seconds
Guesses: 0
Level: Easy
[0, 0, 0, 7, 0, 6, 8, 4, 5, 8, 7, 5, 0, 4, 2, 0, 1, 9, 1, 0, 6, 5, 8, 9, 3, 7, 0, 0, 3, 0, 9, 5, 8, 4, 6, 1, 4, 5, 8, 6, 7, 0, 9, 2, 3, 6, 1, 9, 2, 0, 4, 5, 8, 7, 3, 9, 1, 0, 6, 7, 0, 5, 4, 2, 6, 0, 0, 9, 5, 7, 3, 0, 5, 8, 7, 4, 2, 3, 0, 0, 0]


In [20]:
level = "Easy"
puzzle_long_list = generate_puzzle(level)

Runtime is 1.6252639293670654 seconds
Guesses: 0
Level: Easy


In [21]:
length = len(puzzle_long_list)
n = int(math.sqrt(length))
input_enc = []
for i in range(length):
    el = puzzle_long_list[i]
    if(el != 0):
        name = n * i + el
        input_enc.append([name])

names = get_cell_names(n)
cnf = encode_sudoku(names, input_enc)

Amount of rules 12028
Amount of unique rules 10570


In [25]:
cnf_s="p cnf 729 10570\n"
for line in cnf:
    for rule in line:
        cnf_s+="{} ".format(rule)
    cnf_s=cnf_s[:-1]
    cnf_s+="\n"

cnf_s=cnf_s[:-1]
    
f = open('bla2.cnf','w')
f.write(cnf_s)
f.close()

In [ ]:
solution = pycosat.solve(cnf)
print(decode_sudoku(solution))

In [ ]:
solutions = list(pycosat.itersolve(cnf))

print ('There are in total ', len(solutions), ' possible assignments:')
for sol in solutions[:3]:
    print(decode_sudoku(sol))